# Artificial Neural Networks (ANN)

In [15]:
from os.path import dirname, abspath, join, curdir

import numpy as np
import pandas as pd
import tensorflow as tf

In [16]:
tf.__version__

'2.13.0'

In [17]:
datapath = join(dirname(dirname(abspath(curdir))), 
                        "data", 
                        "raw", 
                        "Churn_Modelling.csv")
datapath

'/Users/a206568245/Projects/MachineLearning/deep-learning/data/raw/Churn_Modelling.csv'

In [18]:
dataset = pd.read_csv(datapath)
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [19]:
X, y

(array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
        [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
        [502, 'France', 'Female', ..., 1, 0, 113931.57],
        ...,
        [709, 'France', 'Female', ..., 0, 1, 42085.58],
        [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
        [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object),
 array([1, 0, 1, ..., 1, 1, 0]))

### Encoding categorical data

In [20]:
# Encode the gender data
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) 
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [21]:
# Perform one-hot encoding on geographical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Split dataset into test/train sets

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature scaling

In [23]:
# Scale all columns by removing the mean and scaling to unit variance, z = (x - u) / s
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train) # scales with mean and std, call on train set ONLY
X_test = sc.transform(X_test) # call transform() since scalar has already been fit with train data
X_train

array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

## Building the ANN

In [24]:
from keras import Input
from keras.models import Sequential
from keras.layers import Dense

# Initialize a Sequential model
ann = Sequential()

In [25]:
# Add the input layer and hidden layers
ann.add(Input(shape=(12,))) # shape = number of feature columns
ann.add(Dense(units=6, activation="relu"))
ann.add(Dense(units=6, activation="relu"))

In [26]:
# Add the output layer
ann.add(Dense(units=1, activation="sigmoid"))

## Training the ANN

In [27]:
# Compile the model
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [28]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5315 - accuracy: 0.7900
Epoch 2/100
250/250 [==============================] - 0s 852us/step - loss: 0.4690 - accuracy: 0.7981
Epoch 3/100
250/250 [==============================] - 0s 814us/step - loss: 0.4423 - accuracy: 0.8084
Epoch 4/100
250/250 [==============================] - 0s 843us/step - loss: 0.4303 - accuracy: 0.8135
Epoch 5/100
250/250 [==============================] - 0s 824us/step - loss: 0.4224 - accuracy: 0.8173
Epoch 6/100
250/250 [==============================] - 0s 810us/step - loss: 0.4166 - accuracy: 0.8186
Epoch 7/100
250/250 [==============================] - 0s 832us/step - loss: 0.4110 - accuracy: 0.8191
Epoch 8/100
250/250 [==============================] - 0s 809us/step - loss: 0.4064 - accuracy: 0.8200
Epoch 9/100
250/250 [==============================] - 0s 898us/step - loss: 0.4018 - accuracy: 0.8230
Epoch 10/100
250/250 [==============================] - 0s 853us/step - los

In [29]:
test = [[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000.0, 2, 1, 1, 50000.0]]
pred = ann.predict(sc.transform(test))
pred > 0.5

1/1 [==============================] - 0s 70ms/step


array([[False]])

In [30]:
y_pred = ann.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

63/63 [==============================] - 0s 805us/step


array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

### Making the Confusion matrix

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[1534,   61],
       [ 210,  195]])

In [32]:
# Get accuracy score
accuracy_score(y_test, y_pred)

0.8645